In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [5]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I’m glad to hear from you. How can I assist you today?


## OK onwards with our first project

In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://stronaosp.pl")
print(ed.title)
print(ed.text)

Strona główna - Strona OSP
Zamów
Zobacz demo
Wersja próbna
Kontakt
Zaznacz stronę
Profesjonalna strona dla OSP
Łatwa w obsłudze strona internetowa, która pomoże Ci zbudować profesjonalny wizerunek OSP, przyciągnąć nowych członków i zaangażować lokalną społeczność.
Zamów stronę
Profesjonalna strona dla OSP
Łatwa w obsłudze strona internetowa, która pomoże Ci zbudować profesjonalny wizerunek OSP, przyciągnąć nowych członków i zaangażować lokalną społeczność.
Zobacz wersję demo
OSP – Serce lokalnej społeczności
Każda Ochotnicza Straż Pożarna jest sercem lokalnej społeczności. Ludzie liczą na Was w trudnych chwilach, ale równie ważne jest, aby znali Wasze działania na co dzień.
Profesjonalna
komunikacja pomaga budować zaufanie, pozyskiwać nowych członków i wzmacniać więź z mieszkańcami. W dzisiejszych czasach strona internetowa to nie tylko wizytówka – to
narzędzie
, które pozwala pokazać, kim jesteście i co robicie, w sposób atrakcyjny i nowoczesny.
Wyzwania w zarządzaniu stroną
Wielu str

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "Jesteś asystentem, który analizuje zawartość strony internetowej \
i zapewnia krótkie podsumowanie, ignorując tekst, który może być związany z nawigacją. \
Odpowiadaj w markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"Przeglądasz stronę internetową zatytułowaną {website.title}"
    user_prompt += "\nZawartość tej strony jest następująca; \
Proszę o krótkie podsumowanie tej strony w markdown. \
Jeśli zawiera wiadomości lub ogłoszenia, również je podsumuj.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
print(user_prompt_for(ed))

Przeglądasz stronę internetową zatytułowaną Strona główna - Strona OSP
Zawartość tej strony jest następująca; Proszę o krótkie podsumowanie tej strony w markdown. Jeśli zawiera wiadomości lub ogłoszenia, również je podsumuj.

Zamów
Zobacz demo
Wersja próbna
Kontakt
Zaznacz stronę
Profesjonalna strona dla OSP
Łatwa w obsłudze strona internetowa, która pomoże Ci zbudować profesjonalny wizerunek OSP, przyciągnąć nowych członków i zaangażować lokalną społeczność.
Zamów stronę
Profesjonalna strona dla OSP
Łatwa w obsłudze strona internetowa, która pomoże Ci zbudować profesjonalny wizerunek OSP, przyciągnąć nowych członków i zaangażować lokalną społeczność.
Zobacz wersję demo
OSP – Serce lokalnej społeczności
Każda Ochotnicza Straż Pożarna jest sercem lokalnej społeczności. Ludzie liczą na Was w trudnych chwilach, ale równie ważne jest, aby znali Wasze działania na co dzień.
Profesjonalna
komunikacja pomaga budować zaufanie, pozyskiwać nowych członków i wzmacniać więź z mieszkańcami. W dzisi

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [11]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, you’re really hitting me with the tough questions, huh? Well, 2 + 2 equals 4. Mind-blowing, I know!


## And now let's build useful messages for GPT-4o-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'Jesteś asystentem, który analizuje zawartość strony internetowej i zapewnia krótkie podsumowanie, ignorując tekst, który może być związany z nawigacją. Odpowiadaj w markdown.'},
 {'role': 'user',
  'content': 'Przeglądasz stronę internetową zatytułowaną Strona główna - Strona OSP\nZawartość tej strony jest następująca; Proszę o krótkie podsumowanie tej strony w markdown. Jeśli zawiera wiadomości lub ogłoszenia, również je podsumuj.\n\nZamów\nZobacz demo\nWersja próbna\nKontakt\nZaznacz stronę\nProfesjonalna strona dla OSP\nŁatwa w obsłudze strona internetowa, która pomoże Ci zbudować profesjonalny wizerunek OSP, przyciągnąć nowych członków i zaangażować lokalną społeczność.\nZamów stronę\nProfesjonalna strona dla OSP\nŁatwa w obsłudze strona internetowa, która pomoże Ci zbudować profesjonalny wizerunek OSP, przyciągnąć nowych członków i zaangażować lokalną społeczność.\nZobacz wersję demo\nOSP – Serce lokalnej społeczności\nKażda Ochotnicza Straż Pożar

## Time to bring it together - the API for OpenAI is very simple!

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://stronaosp.pl")

'# Podsumowanie Strony OSP\n\n**Opis ogólny:**  \nStrona oferuje profesjonalne rozwiązania internetowe dla Ochotniczych Straży Pożarnych (OSP), które chcą zbudować swój wizerunek, przyciągnąć nowych członków oraz angażować lokalną społeczność. Umożliwia łatwe zarządzanie treściami oraz dostarcza narzędzi do komunikacji i prezentacji działań OSP.\n\n**Kluczowe funkcje i zalety:**\n- Intuicyjna obsługa i łatwe zarządzanie treściami.\n- Możliwość publikacji informacji o akcjach, wydarzeniach i ogłoszeniach.\n- Statystyki odwiedzin i aktywności jednostki.\n- Dostosowanie wyglądu strony do potrzeb OSP.\n- Możliwość budowania silnych więzi z lokalną społecznością i sponsorami.\n- Wsparcie w pozyskiwaniu funduszy na zakup strony.\n\n**Wyzwania:**  \nWiele OSP zmaga się z trudnościami w utrzymaniu profesjonalnego wizerunku online, co prowadzi do braku aktualizacji stron i ich mniejszej frekwencji.\n\n**Dlaczego warto mieć stronę internetową?**  \nPosiadając stronę, OSP mogą utrzymać pełną kont

In [17]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://stronaosp.pl")

# Podsumowanie Strony OSP

Strona OSP oferuje profesjonalne i łatwe w obsłudze rozwiązania internetowe dla Ochotniczych Straży Pożarnych. Jej główne założenia to:

- **Budowanie Wizerunku:** Strona ma pomóc OSP w stworzeniu profesjonalnego wizerunku online, co sprzyja pozyskiwaniu nowych członków i angażowaniu lokalnej społeczności.
- **Dedykowane Funkcje:** Oferowane są funkcje takie jak galerie zdjęć, harmonogramy wydarzeń, formularze rejestracyjne oraz miejsce na publikowanie osiągnięć, co umożliwia efektywną komunikację i prezentację działań OSP.
- **Zarządzanie i Statystyki:** Umożliwia łatwe zarządzanie treściami oraz dostęp do statystyk dotyczących odwiedzin i aktywności jednostki.
- **Wsparcie Finansowe:** Strona oferuje gotowe pisma do skierowania do urzędów gminy i sponsorów w celu pozyskania funduszy na utworzenie i utrzymanie strony.

## Kluczowe Zalety
- **Niezależność i Kontrola:** Użytkownicy mają pełną kontrolę nad treściami oraz niezależność od mediów społecznościowych.
- **Integracja z Lokalną Społecznością:** Strona umożliwia budowanie relacji z mieszkańcami, zwiększając ich zaangażowanie.
- **Edukacja:** Możliwość publikowania artykułów i materiałów edukacyjnych dotyczących bezpieczeństwa i działań ratowniczych.

Strona jest odpowiedzią na wyzwania, przed którymi stoją OSP w kontekście zarządzania swoją obecnością w sieci oraz efektywnej komunikacji z lokalną społecznością.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [19]:
display_summary("https://cnn.com")

# Podsumowanie strony CNN

Strona CNN dostarcza najnowsze wiadomości z różnych dziedzin, w tym polityki, zdrowia, biznesu, sportu oraz wydarzeń międzynarodowych. Oto kluczowe informacje znalezione w treści:

## Najnowsze wiadomości:
- **Protesty przeciw Trumpowi i Muskowi**: W odpowiedzi na to, co organizatorzy nazywają „wrogim przejęciem”, miliony ludzi wyszły na ulice w wszystkich 50 stanach USA oraz na całym świecie.
- **Atak Rosji**: Ostatni atak na Kryvyi Rih w Ukrainie doprowadził do śmierci 19 osób, w tym 9 dzieci, co czyni go jednym z najgorszych incydentów w tym roku.
- **Pojawienie się Papieża Franciszka**: Papież Franciszek zaskoczył publiczność swoją pierwszą publiczną obecnością po hospitalizacji.
- **Analiza tarfów**: Niespodziewana reakcja na taryfy Trumpa i ich wpływ na inflację i rynek pracy w USA.

## Inne istotne wiadomości:
- **Zmarł 108-letni bohater II wojny światowej**: Śmierć jednego z legendarnych "Triple Nickles".
- **Dziecięce wspomnienie trudnej historii**: Nowe badania pokazują, jak genotyp mumi w Saharze związany jest z historią regionu.
- **Inne istotne wydarzenia**: Wydano oświadczenia dotyczące podróży, wpływu przez sztuczną inteligencję na młodzież i problemy zdrowotne związane z podróżami.

Strona nie tylko informuje o bieżących wydarzeniach, ale także angażuje użytkowników w dialog dotyczący treści oraz jakości reklamowych.

In [20]:
display_summary("https://anthropic.com")

# Podsumowanie strony Home | Anthropic

Strona przedstawia firmę Anthropic, koncentrującą się na badaniach i rozwoju rozwiązań AI z naciskiem na bezpieczeństwo. 

## Nowości i Aktualizacje
- **Claude 3.7 Sonnet**: Najnowszy model AI (hybrydowy model rozumowania) udostępniony użytkownikom.
- **Claude Code**: Nowe narzędzie wspierające programowanie, pozwalające na tworzenie aplikacji wspieranych przez AI.

## Funkcje i Usługi
- **API i Rozwiązania**: Możliwość budowania aplikacji i doświadczeń związanych z AI.
- **Plany subskrypcyjne dla różnych użytkowników**: Oferowane są plany dla firm oraz edukacji, a także dostęp do aplikacji Claude.

## Badania i Zobowiązania
- Organizacja prowadzi badania w dziedzinie AI, koncentrując się na bezpieczeństwie i odpowiedzialności technologii.
- Zespół składa się z ekspertów z różnych dziedzin, takich jak ML, fizyka i polityka.

## Ogłoszenia
- Wydano dokumenty dotyczące polityki i zaangażowania w bezpieczeństwo AI, w tym "Core Views on AI Safety" z 8 marca 2023 roku.

Strona podkreśla misję Anthropic w zakresie tworzenia niezawodnych i korzystnych systemów AI.

In [21]:
display_summary("https://strazak.online/")

# Podsumowanie strony Strażak.Online

**Ogólny opis:**
Strażak.Online to aplikacja stworzona dla Ochotniczych Straży Pożarnych (OSP), która ma na celu ułatwienie ewidencji, zarządzania dokumentacją i usprawnienie komunikacji między członkami jednostki. Umożliwia szybkie przygotowywanie wniosków, raportów i zestawień, a także integruje funkcje zarządzania terminami i wydarzeniami.

**Kluczowe funkcje:**
- **Ewidencja danych:** Obejmuje 11 modułów, które pozwalają na efektywne zarządzanie dokumentacją.
- **Generowanie dokumentów:** Umożliwia szybkie tworzenie wniosków i zestawień w oparciu o zebrane dane.
- **Kalendarz wydarzeń:** Wszechstronny kalendarz dla jednostki, aby nie przegapić istotnych spotkań i terminów.
- **System ról i uprawnień:** Pozwala na kontrolowanie dostępu do danych w zależności od przydzielonych ról.
- **Obsługa na różnych urządzeniach:** Z aplikacji można korzystać na komputerach, tabletach i smartfonach, co zwiększa jej dostępność.
- **Wsparcie techniczne:** Użytkownicy mogą korzystać z systemu zgłoszeń w celu uzyskania pomocy.

**Opinie użytkowników:**
1. **OSP KSRG Przygodzice:** Użytkownik chwali aplikację za uproszczenie procesów administracyjnych związanych z prowadzeniem dokumentacji.
2. **OSP Słomczyn:** Ulepszenia w administracji i skracanie czasu potrzebnego na rozliczanie zdarzeń.
3. **OSP Kęty Podlesie:** Zadowolenie z możliwości pracy na więcej niż jednej jednostce.
4. **OSP Ostrówek:** Komfort korzystania z aplikacji oraz koncentracja na najważniejszych zadaniach.

**Bezpieczeństwo:** Strona podkreśla wysokie standardy bezpieczeństwa danych, w tym dwuetapowe uwierzytelnianie oraz odpowiednie zarządzanie dostępem wewnętrznym.

**Dostępność:** Możliwość przetestowania aplikacji przez miesiąc bezpłatnie, co pozwala na ocenę jej funkcjonalności przed podjęciem decyzji o zakupie.

Strona promuje nowoczesne podejście do zarządzania OSP i zachęca do zakupu licencji, oferując jednocześnie wygodne formy wsparcia.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

In [22]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(

SyntaxError: invalid syntax (3985043996.py, line 15)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293